In [19]:
# Import the custom library
import logic_algorithms as la
import sympy as sp
from sympy.logic.boolalg import Implies, And, Or, Not, Equivalent

# Define the plausability table

In [20]:
# Define the table with the plausibility orders
plausibility_table = {
    'p, q': {'states': ['x'], 'order': 2},
    'p, n_q': {'states': ['y'], 'order': 1},
    'n_p, q': {'states': ['z'], 'order': 2},
    'n_p, n_q': {'states': ['w'], 'order': 3}
}

# Print the table to verify
for key, value in plausibility_table.items():
    print(f"{key}: {value}")


p, q: {'states': ['x'], 'order': 2}
p, n_q: {'states': ['y'], 'order': 1}
n_p, q: {'states': ['z'], 'order': 2}
n_p, n_q: {'states': ['w'], 'order': 3}


## Contraction

In [21]:
# Results of the contraction
print("Contraction with p: " + str(la.contraction(plausibility_table, 'p')))
print("Contraction with q: " + str(la.contraction(plausibility_table, 'q')))
print("Contraction with n_p: " + str(la.contraction(plausibility_table, 'n_p')))
print("Contraction with n_q: " + str(la.contraction(plausibility_table, 'n_q')))

Contraction with p: (['y'], ['z'])
Contraction with q: (['x'], ['y'])
Contraction with n_p: (['z'], ['y'])
Contraction with n_q: (['y'], ['x'])


## Revision

In [22]:
# Results of the revision
print("Revision with p: " + str(la.revision(plausibility_table, 'p')))
print("Revision with q: " + str(la.revision(plausibility_table, 'q')))
print("Revision with n_p: " + str(la.revision(plausibility_table, 'n_p')))
print("Revision with n_q: " + str(la.revision(plausibility_table, 'n_q')))

Revision with p: ['y']
Revision with q: ['x']
Revision with n_p: ['z']
Revision with n_q: ['y']


# Belief Base

# Truth table

In [23]:
# Define the symbols
p, q, r, s = sp.symbols('p q r s')

# Define the formulas in the knowledge base (KB)
KB = [
    Implies(Not(p), q),  # ¬p → q
    Implies(q, p),       # q → p
    Implies(p, And(r, s)),  # p → r ∧ s
    And(p, And(r, s))  # p ∧ r ∧ s
]

# Compute and display the truth table for the given knowledge base
la.compute_truth_table([p, q, r, s], KB)

,p,q,r,s,"Implies(~p, q)","Implies(q, p)","Implies(p, r & s)",p & r & s
0,F,F,F,F,F,T,T,F
1,F,F,F,T,F,T,T,F
2,F,F,T,F,F,T,T,F
3,F,F,T,T,F,T,T,F
4,F,T,F,F,T,F,T,F
5,F,T,F,T,T,F,T,F
6,F,T,T,F,T,F,T,F
7,F,T,T,T,T,F,T,F
8,T,F,F,F,T,T,F,F
9,T,F,F,T,T,T,F,F


## Revision of a Belief Base

In [24]:
# Define the symbols
s, q, r = sp.symbols('p q r')

# Define a belief base
belief_base = {sp.Equivalent(r, sp.Or(p, s))}

# New belief
new_belief = q

# Check if adding the new belief keeps the belief base consistent
belief_base_with_new = belief_base | {new_belief}

if la.is_consistent(belief_base_with_new):
    print("The belief base with the new belief is CONSISTENT.")
    belief_base = belief_base_with_new
else:
    print("The belief base with the new belief is INCONSISTENT. Applying contraction...")
    # Apply contraction
    inconsistent_belief = sp.Or(p, q)
    contracted_bases = la.bb_contraction(belief_base, inconsistent_belief)
    belief_base = set(contracted_bases[0])
    belief_base.add(new_belief)

# Display the new belief base
print("New belief base:")
for belief in belief_base:
    print(belief)

The belief base with the new belief is CONSISTENT.
New belief base:
q
Equivalent(p, r)


## Revision of formulas

In [25]:
# Define the symbols
p, q = sp.symbols('p q')

# Define the formulas
formulas = {
    'p -> q': sp.Implies(p, q),
    'q -> p': sp.Implies(q, p),
    'p ∧ q': sp.And(p, q),
    'p ∨ q': sp.Or(p, q),
    'p ↔ q': sp.Equivalent(p, q)
}

# Given belief set revision with ¬p
belief_revision = sp.Not(p)

# Check each formula against the revised belief set
results = {}
for formula, expr in formulas.items():
    # Substitute ¬p into the expression
    revised_expr = expr.subs(p, False)
    # Evaluate the revised expression
    result = sp.simplify(revised_expr)
    results[formula] = result

# Print the results
print("Results of the belief revision with ¬p (simplification):")
for formula, result in results.items():
    print(f"{formula}: {result}")

# Compute and display the belief revision table
styled_table = la.compute_belief_revision_table(formulas, belief_revision)
styled_table


Results of the belief revision with ¬p (simplification):
p -> q: True
q -> p: ~q
p ∧ q: False
p ∨ q: q
p ↔ q: ~q


,p,q,Belief Revision: ¬p,p -> q,q -> p,p ∧ q,p ∨ q,p ↔ q,After ¬p: p -> q,After ¬p: q -> p,After ¬p: p ∧ q,After ¬p: p ∨ q,After ¬p: p ↔ q
0,F,F,T,T,T,F,F,T,T,T,F,F,T
1,F,T,T,T,F,F,T,F,T,F,F,T,F
2,T,F,F,F,T,F,T,F,T,T,F,F,T
3,T,T,F,T,T,T,T,T,T,F,F,T,F


## Contraction

In [26]:
# Define the symbols
p, q = sp.symbols('p q')

# Define the belief base and the proposition
# belief_base = {p, q, Implies(p, q), Implies(Not(p), q)}
# proposition = q

belief_base = {p, sp.Or(p,q), sp.And(p, q), sp.Equivalent(p, q)}
proposition = p

result = la.bb_contraction(belief_base, proposition)
for i, subset in enumerate(result):
    print(f"Maximal subset {i+1}: {set(subset)}")

Maximal subset 1: {Equivalent(p, q)}
Maximal subset 2: {p | q}


# Convert to CNF

In [27]:
# Define the symbols
a, b, c, d, e, f = sp.symbols('a b c d e f')

# Define a list of logical expressions
expressions = [
    sp.Equivalent(a, sp.Or(c, e)),
    sp.Implies(e, d),
    sp.Implies(sp.And(b, f), sp.Not(c)),
    sp.Implies(e, c),
    sp.Implies(c, f),
    sp.Implies(c, b),
]

# Convert the expressions to CNF
cnf_expressions = la.convert_to_cnf(expressions)

# Display the original and CNF expressions
for i, expr in enumerate(expressions):
    print(f"Original Expression {i+1}: {expr}")
    print(f"CNF {i+1}: {cnf_expressions[i]}")

Original Expression 1: Equivalent(a, c | e)
CNF 1: (a | ~c) & (a | ~e) & (c | e | ~a)
Original Expression 2: Implies(e, d)
CNF 2: d | ~e
Original Expression 3: Implies(b & f, ~c)
CNF 3: ~b | ~c | ~f
Original Expression 4: Implies(e, c)
CNF 4: c | ~e
Original Expression 5: Implies(c, f)
CNF 5: f | ~c
Original Expression 6: Implies(c, b)
CNF 6: b | ~c
